In [2]:
import os, pandas as pd
import datetime as dt
nps_t1data = os.path.join(os.path.pardir,'data','raw','T1.txt')
nps_t1csv = os.path.join(os.path.pardir,'data','interim','T1.csv')
nps_t2data = os.path.join(os.path.pardir,'data','raw','T2.txt')
nps_t2csv = os.path.join(os.path.pardir,'data','interim','T2.csv')

In [3]:
def get_SI(NetGain, SIGain):
    '''
    finds the best SI rate by apporximating to the mid point
    Input : NetGain is the difference of Current Value of investement and Net Investment
            SIGain is the SI value without the Rate of interest i.e. Principle * Time / 100
    Returns : Simple Interest Rate that will give the returns as NetGain
    '''
    min_int = 0
    max_int = 200
    while ( True ):
        mid_int = (max_int + min_int) /2
        #print("Internest rate .. {}".format(mid_int))
        if abs(NetGain - (SIGain * mid_int)) < 100:
            return mid_int
        elif NetGain < SIGain * mid_int:
            max_int = mid_int
        else:
            min_int = mid_int

def get_data(raw_data_file):
    '''
    Converts the data file to a dataFrame. The format of raw file is 
        Date Amount TransactionType 
    The above fileds are space sepatated and it fetched from NPS page
    '''
    ## Create a list with the file data feed
    t2data = list()
    with open(raw_data_file) as datafile:
        for line in datafile:
            t2data.append(line.split("\t"))
            
    ## Last value is just summation, so ignore it
    df = pd.DataFrame(t2data[1:-1],columns=t2data[0])
      
    ## Store current value in all rows
    df['CurrentValue'] = float(t2data[-1][1]) #* -1

    ## Export as csv
    #df.to_csv('nps_t2csv', index=False, header=True)

    ## Convert values to numeric type
    df[["AMOUNT", "CurrentValue"]] = df[["AMOUNT", "CurrentValue"]].apply(pd.to_numeric)

    ## Negate Current Value as it will be -ve   
    df[["CurrentValue"]] = df[["CurrentValue"]] * -1

    ## Date is in string
    df['DATE'] =  pd.to_datetime(df['DATE'], format='%d-%b-%Y')

    ## Calulate days till now from the investment date
    df['DaysTillNow'] = (dt.datetime.now() - df['DATE']).dt.days

    ## SI gain is without rate of interest.
    df['SI'] = df['AMOUNT'] * df['DaysTillNow'] /36500
    
    return df

def get_current_value(raw_data_file):
    ## Collect summation value and date in the last row
    return float(t2data[-1][1]) * -1
    ##curret_value_on = t2data[-1][0]
    
dfrs = pd.DataFrame(['Invested','Simpe Interest','Current value','Net Gain'],columns=(['Description']))

In [4]:
df1 = get_data(nps_t1data)

SI1 = get_SI((df1['CurrentValue'][0] - df1['AMOUNT'].sum()), df1['SI'].sum())
df1['SI_gain'] = SI1 * df1['SI']

dfrs['Tier1'] = [df1['AMOUNT'].sum(),SI1, df1['CurrentValue'][0] , df1['SI_gain'].sum()]

df1.head()

,DATE,AMOUNT,TRANSACTION TYPE,CurrentValue,DaysTillNow,SI,SI_gain
0,2010-04-20,455.88,Contribution\n,743405.73,3364,42.015899,512.889394
1,2010-05-13,3977.94,Contribution\n,743405.73,3341,364.117741,4444.796641
2,2010-08-16,3977.94,Contribution\n,743405.73,3246,353.764198,4318.410624
3,2010-11-15,3977.94,Contribution\n,743405.73,3155,343.846595,4197.346125
4,2011-02-14,3977.94,Contribution\n,743405.73,3064,333.928991,4076.281625


In [5]:
df2 = get_data(nps_t2data)

SI2 = get_SI((df2['CurrentValue'][0] - df2['AMOUNT'].sum()), df2['SI'].sum())
df2['SI_gain'] = SI2 * df2['SI']

dfrs['Tier2'] = [df2['AMOUNT'].sum(),SI2, df1['CurrentValue'][0], df2['SI_gain'].sum()]

df2.head()

,DATE,AMOUNT,TRANSACTION TYPE,CurrentValue,DaysTillNow,SI,SI_gain
0,2010-04-20,977.94,Contribution\n,2320427.36,3364,90.131237,1030.920036
1,2010-05-18,4977.94,Contribution\n,2320427.36,3336,454.970078,5203.942418
2,2010-08-18,4977.94,Contribution\n,2320427.36,3244,442.422941,5060.428419
3,2010-11-22,4977.94,Contribution\n,2320427.36,3148,429.330277,4910.674680
4,2011-02-21,4977.94,Contribution\n,2320427.36,3057,416.919523,4768.720615


In [8]:
# Display details for Tier1 & Tier2 account
dfrs.round({'Tier1':2, 'Tier2':2})

,Description,Tier1,Tier2
0,Invested,575141.76,2019642.56
1,Simpe Interest,12.21,11.44
2,Current value,743405.73,743405.73
3,Net Gain,168164.78,300871.99
